## Sampling fot the Match Filter

In [ ]:
#Libraries to import
using SerialPorts
using PyPlot
using FFTW 

In [ ]:
list_serialports() # show available ports
try
    ser = SerialPort("COM3:", 9600) 

    r = readavailable(ser)

    # Start a conversion
    write(ser, "c")

    BytesAvailable = bytesavailable(ser)

    while bytesavailable(ser) < 1
        continue 
    end
    sleep(0.05)
    r = readavailable(ser)
    t = r
    # Get the values
    write(ser, "p") # Print DMA buffer
    while bytesavailable(ser) < 1
        continue # wait for a response
    end
    a = []
    b = ""
    i = 0
    while true
        if bytesavailable(ser) < 2
            sleep(0.005) # Wait and check again
            if bytesavailable(ser) < 1
                break
            end
        end

        b = string(b, readavailable(ser))

    end 

    close(ser)

    a=split(b, ("\r\n"))

    println(length(a))
    
catch e
    println("something bad happened")
end

In [ ]:
ac = zeros(Int,length(a))

for i = 1:length(a)-1
    ac[i] = parse(Int,chomp(a[i]))
end

match = (3.3/4096).*ac

figure()
plot(match)

## Sampling the Echo 

In [ ]:
list_serialports() # show available ports
try
    ser = SerialPort("COM3:", 9600) 

    r = readavailable(ser)

    # Start a conversion
    write(ser, "c")

    BytesAvailable = bytesavailable(ser)

    while bytesavailable(ser) < 1
        continue 
    end
    sleep(0.05)
    r = readavailable(ser)
    
    # Get the values
    write(ser, "p") # Print DMA buffer
    while bytesavailable(ser) < 1
        continue # wait for a response
    end
    a = []
    b = ""
    i = 0
    while true
        if bytesavailable(ser) < 2
            sleep(0.005) # Wait and check again
            if bytesavailable(ser) < 1
                break
            end
        end

        b = string(b, readavailable(ser))

    end 

    close(ser)

    a=split(b, ("\r\n"))

    println(length(a))
    
catch e
    println("something bad happened")
end

In [ ]:

rc = zeros(Int,length(a))

for i = 1:length(a)-1
    rc[i] = parse(Int,chomp(a[i]))
end

receive = (3.3/4096).*rc

figure()
plot(match)

In [ ]:
v_tx = match

print(length(v_tx))
figure() # Create a new figure
title("Transmitted pulse")
xlabel("Samples")
plot(v_tx) # Basic plot, axis labeled in samples

In [ ]:
#to see the freqeuncy domain use FFT function
V_TX = fft(v_tx);

figure()
title("FFT of transmitted pulse on sampling axis")
xlabel("Samples")
plot( abs.(V_TX) )

In [ ]:
v_rx = received


figure() # Create a new figure
title("Unprocessed received pulse")
plot(v_rx) # Basic plot, axis labeled in samples

In [ ]:
V_RX = fft(v_rx);

figure()
title("FFT of received pulse before processing")
plot( abs.(V_RX) )

In [ ]:
H = conj( V_TX);

V_MF = H.*V_RX;
v_mf = ifft(V_MF);

figure() # Create a new figure
xlabel("Time in seconds")
title(" Match Filtered output of received pulse")
plot(v_mf) # Basic plot, axis labeled in samples

figure()
plot( abs.(V_MF) )

In [ ]:
V_ANAL = 2*V_MF; # make a copy and double the values
N = length(V_MF);

V_ANAL = 2*V_MF; # make a copy and double the values

N = length(V_MF);

if mod(N,2)==0 # case N even
    neg_freq_range = Int(N/2):N; # Define range of “neg-freq” components
else # case N odd
    neg_freq_range = Int((N+1)/2):N;
end

V_ANAL[neg_freq_range] .= 0; # Zero out neg components in 2nd half of array.
v_anal = ifft(V_ANAL);

In [ ]:
figure()
title("Analytic signal")
xlabel("Range in meters")
plot(r,abs.(v_anal))

In [ ]:
j=im; # Assign j as sqrt(-1) (“im” in julia)
v_bb = v_anal.*exp.(-j*2*pi*f0.*t);

In [ ]:
figure()
subplot(2,1,1)
plot(r,abs.(v_bb))
subplot(2,1,2)
plot(r,angle.(v_bb)) #query

In [ ]:
V_BB = fft(v_bb); 
figure()
plot(abs.(v_bb))